In [ ]:
import os
import shutil
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.preprocessing import image

In [ ]:
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
inception_model.trainable = False

In [ ]:
model = Sequential()
model.add(inception_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      zoom_range=0.2,
                      horizontal_flip=True,
                      fill_mode='nearest')


In [ ]:
train_generator = train_datagen.flow_from_directory(
                    'train/',
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
                          'validation/',
                          target_size=(150, 150),
                          batch_size=20,
                          class_mode='binary')

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])

In [ ]:
history = model.fit_generator(
              train_generator,
              steps_per_epoch=100,
              epochs=50,
              validation_data=validation_generator,
              validation_steps=50)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('camila_ariana.h5')

In [ ]:
def read_and_process_image(directory, list_of_images):
    X = [] # images
    
    for images in list_of_images:
        img = image.load_img('test/'+directory+'/'+images, target_size=(150, 150))
        x = image.img_to_array(img)
        X.append(x)
    
    return X

In [ ]:
test_images_ariana = os.listdir("test/ariana")
test_images_camilia = os.listdir("test/camila")

In [ ]:
X_ariana = read_and_process_image('ariana', test_images_ariana[80:90])
X_camila = read_and_process_image('camila', test_images_camilia[90:100])
X = np.concatenate((X_ariana, X_camila), axis=0)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
i = 0
columns = 10
text_labels = []
plt.figure(figsize=(30,20))
for batch in test_datagen.flow(X[:10], batch_size=1):
    pred = model.predict(batch)
    if pred > 0.5:
        text_labels.append('camila')
    else:
        text_labels.append('ariana')
    plt.subplot(5 / columns + 1, columns, i + 1)
    plt.title('This is a ' + text_labels[i])
    imgplot = plt.imshow(batch[0])
    i += 1
    if i % 10 == 0:
        break
plt.show()

In [ ]:
i = 0
columns = 10
text_labels = []
plt.figure(figsize=(30,20))
for batch in test_datagen.flow(X[10:], batch_size=1):
    pred = model.predict(batch)
    if pred > 0.5:
        text_labels.append('camila')
    else:
        text_labels.append('ariana')
    plt.subplot(5 / columns + 1, columns, i + 1)
    plt.title('This is a ' + text_labels[i])
    imgplot = plt.imshow(batch[0])
    i += 1
    if i % 10 == 0:
        break
plt.show()